# Covid Use Case Step 4 Targeted Attack with nVITA

In [1]:
import argparse
import os
import time
from pathlib import Path

import numpy as np
import nvita.models.train as mt
import torch
from nvita.attacks.rnvita import RNVITA
from nvita.attacks.utils import (append_result_to_csv_file,
                                 check_result_file_path,
                                 create_empty_result_csv_file)
from nvita.models.data import SplittedTSData
from nvita.models.utils import load_model
from nvita.utils import open_json

d:\Programming\IDE\Anaconda\envs\hons\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()

my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))

DF_NAME = "CovidUSA"
SEED = int(my_metadata["seeds"][0])
RESTRICTED_F_INDS = [0, 7, 8]
# features indices that do not allow to be attacked. 

In [3]:
def restrict_perturbed_features(window_range, restricted_f_inds):
    """
    Setup those features that do not allow to be attacked. 
    """
    result = torch.clone(window_range)
    for f_ind in restricted_f_inds:
        result[f_ind] = 0 
    return result

def get_eta_stats(eta, n):
    """
    Calculate attack statistics for nvita
    """
    attack_stats = []
    for time_ind in range(eta.shape[1]):
        for feature_ind in range(eta.shape[2]):
            current_val = eta[0, time_ind, feature_ind]
            if current_val != 0:
                attack_stats.append((time_ind, feature_ind, int(np.sign(current_val).item())))
                
    if n < len(attack_stats):
        print("Attack Value Number Warning!!!")
    return attack_stats

## Main function for experiments of CovidUSA

In [4]:
def run_use_case_exp(df_name, seed, epsilon, n, target, demo = None):
    """
    main function that runs the targeted experiments for nvita on CovidUSA and record the results
    """
    s_data = SplittedTSData()
    s_data = s_data.load_splitted_data(PATH_ROOT, df_name, seed)
    # load data
    m = load_model(PATH_ROOT, df_name, seed, "LSTM")
    # load model
    attack_name = "Targeted_" + str(n) + "VITA"
    if demo == None:
        max_w_size = s_data.X_test.shape[0]
    else:
        max_w_size = demo

    path_out_dir = os.path.join(PATH_ROOT, "examples", "covidusa_results")
    path_out_file = os.path.join(path_out_dir, "attack_"+attack_name+"_target_"+target+".csv")

    first_result_line_list = ["Epsilon", "Target Direction", "Test Index", "Attack Name", "True y", "Original y Pred", "Attacked y Pred", "Attacked AE", "Original AE","Eta", "Adv Example", "Percentage Changeed from the Attack", "Attack Stats" ]

    create_empty_result_csv_file(path_out_file, first_result_line_list)
    # Create empty csv file with the column names

    for test_ind in range(max_w_size):
        
        X_current = torch.reshape(s_data.X_test[test_ind], s_data.single_X_shape)
        ground_truth_y = torch.reshape(s_data.y_test[test_ind], s_data.single_y_shape)
        window_range = s_data.window_ranges[test_ind]
        #window_range = restrict_perturbed_features(s_data.window_ranges[test_ind], RESTRICTED_F_INDS)
        mean_pred, std_pred = mt.get_mean_and_std_pred(m, X_current)

        if target == "Positive":
            attack_goal = 2
        elif target == "Negative":
            attack_goal = -1
        attack_goal_tup = (test_ind, attack_goal)

        att = RNVITA(n, epsilon, m, targeted=True)
        X_adv, _ = att.attack(X_current, attack_goal_tup, window_range, RESTRICTED_F_INDS, maxiter=600, seed=seed, tol=0.0001)
        
        adv_y_pred = mt.adv_predict(m, X_adv)
        eta = X_adv - X_current
        original_y_pred = mt.adv_predict(m, X_current)
        adv_y_pred = mt.adv_predict(m, X_adv)
        original_ae = np.absolute(original_y_pred - attack_goal)
        attacked_ae = np.absolute(adv_y_pred - attack_goal)

        percentage = ((adv_y_pred - original_y_pred) / s_data.window_ranges[test_ind][s_data.y_ind]).item() * 100
        attack_stats = get_eta_stats(eta, n)

        result = [epsilon, target, test_ind, str(att), str(ground_truth_y.item()), original_y_pred, str(adv_y_pred), attacked_ae, original_ae, str(eta.tolist()).replace(",", ";"), str(X_adv.tolist()).replace(",", ";"), str(percentage), str(attack_stats).replace(",", ";")]
        append_result_to_csv_file(path_out_file, result)
        
    print('Save to:', path_out_file)


## Run 1, 3, 5VITA for both positive and negative targeted attacks

In [6]:
for n in [1,3,5]:
    for target in ["Positive", "Negative"]:
        run_use_case_exp(DF_NAME, SEED, 0.2, n, target)

IndentationError: expected an indented block (1243552316.py, line 3)